# PyTorch workflow

Reference: https://www.learnpytorch.io/01_pytorch_workflow/



![workflow](https://raw.githubusercontent.com/mrdbourke/pytorch-deep-learning/main/images/01_a_pytorch_workflow.png)

## 1. Create a dataset

We will use the MNIST dataset

In [ ]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(X_train[0])
print(y_train[0])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

class MyDataSet(torch.utils.data.Dataset):
  def __init__(self, X, y):
    super(MyDataSet, self).__init__()
    self._X = (X/255).astype('float32')
    self._y = y

  def __len__(self):
    return self._X.shape[0]

  def __getitem__(self, idx):
    _X = self._X[idx]
    _y = self._y[idx]
    return _X, _y

## `Dataloader` allows us to split the data into minibatches

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 10

dataset = MyDataSet(X_train, y_train)
train_set, val_set = torch.utils.data.random_split(dataset, [50000, 10000])

trainloader = DataLoader(train_set, batch_size=64, shuffle=True)
valloader = DataLoader(val_set, batch_size=64, shuffle=False)

## 2. Construct a neural network with four layers:
* Input layer: 784 nodes
* First hidden layer: 512 nodes
* Second hidden layer: 512 nodes
* Output layer: 10 nodes

In [ ]:
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # flatten from 2-dim image to a 1-dim vector
        self.flatten = nn.Flatten()
        # attributes of layers
        self.lin1 = nn.Linear(28*28, 512)
        self.act1 = nn.ReLU()
        self.lin2 = nn.Linear(512, 512)
        self.act2 = nn.ReLU()
        self.lin3 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.flatten(x) # x has shape (64, 28*28)
        # define the network using attributes defined above
        x = self.lin1(x)
        x = self.act1(x)
        x = self.lin2(x)
        x = self.act2(x)
        x = self.lin3(x)
        return x


model = SimpleNN()

## 2.1 Specify the loss function and optimization algorithm

In [ ]:
loss_fn = nn.CrossEntropyLoss()

# Notice that the optimizer stores the model's parameters
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## 2.2 Build training and validation loop

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        # Compute prediction
        pred = model(X)
        # Compute loss
        loss = loss_fn(pred, y)

        # Clear the gradient first
        optimizer.zero_grad()
        # Compute the gradient with backpropagation
        loss.backward()
        # Update parameters with the optimizer
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def val_loop(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    val_loss = 0
    correct = 0

    with torch.no_grad():
        for X, y in dataloader:
            # Compute prediction
            pred = model(X)
            # Accumulate the loss
            val_loss += loss_fn(pred, y)
            correct += (pred.argmax(1) == y).sum().item()

    val_loss /= num_batches
    correct /= size
    print(f"Validation Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {val_loss:>8f} \n")

## 3. Fit the model to the data

In [ ]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(trainloader, model, loss_fn, optimizer)
    val_loop(valloader, model, loss_fn)
print("Done!")

## 4. Make predictions

In [ ]:
val_loop(valloader, model, loss_fn)

# Exercise

Note: you may finish the exercise in a new notebook.

Explore the file `CIFAR-10` dataset, which contains the data of images classified into **10 classes**.

1. First, split your training set into 80% training set and 20% validation set

2. Design your own neural network.

3. Try to get your **validation** accuracy as high as possible by trying different network designs, learning rates, batch sizes and epochs.

4. Report the accuracy of your final model's predictions on the test set.

In [ ]:
!pip install cifar10

In [ ]:
from keras.datasets import cifar10

(train_x, train_y), (test_x, test_y) = cifar10.load_data()

In [ ]:
plt.imshow(train_x[1])
print(train_y[1])

#### See the class names [here](https://www.tensorflow.org/api_docs/python/tf/keras/datasets/cifar10/load_data#the_classes_are).